# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

In [ ]:
# Setting up the PySpark environment

# Install java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

# Unzip file
!tar -xf spark-3.1.2-bin-hadoop3.2.tgz

# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

# Add environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

# findspark will locate spark in the system
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,294 kB]
Get:13

#### Start a simple Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Hands-on PySpark on Google Colab") \
        .getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = "/content/drive/MyDrive/IYKRA/#10/Task_1/walmart_stock.csv"
spark_df = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath)

In [ ]:
spark_df.dtypes

[('Date', 'string'),
 ('Open', 'double'),
 ('High', 'double'),
 ('Low', 'double'),
 ('Close', 'double'),
 ('Volume', 'int'),
 ('Adj Close', 'double')]

#### What are the column names?

In [ ]:
spark_df.schema.names

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [ ]:
spark_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [ ]:
spark_df.select(spark_df.columns[:5]).show()

+----------+------------------+------------------+------------------+------------------+
|      Date|              Open|              High|               Low|             Close|
+----------+------------------+------------------+------------------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18|
|2012-01-10|             59.43|59.709998999999996|             58.98|59.040001000000004|
|2012-01-11|         59.060001|         59.529999|59.040001000000004|         59.400002|
|2012-01-12|59.790001000000004|              60.0|         59.400002|              59.5|
|2012-01-13|         

#### Use describe() to learn about the DataFrame.

In [ ]:
spark_df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar.

If you get stuck on this, don't worry, just find the way :)

In [ ]:
from pyspark.sql.functions import format_number
desc_df = spark_df.describe()
desc_df.select(desc_df['summary'],
              format_number(desc_df['Open'].cast('float'),2).alias('Open'),
              format_number(desc_df['High'].cast('float'), 2).alias('High'),
              format_number(desc_df['Low'].cast('float'), 2).alias('Low'),
              format_number(desc_df['Close'].cast('float'), 2).alias('Close'),
              desc_df['Volume'].cast('int').alias('Volume'),
              format_number(desc_df['Adj Close'].cast('float'), 2).alias('AdjClose')).show()

+-------+--------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|AdjClose|
+-------+--------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|   67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|    6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|   50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|   84.91|
+-------+--------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
df = spark_df.withColumn("HV Ratio", spark_df['High']/spark_df['Volume'])
df.select("HV Ratio").show(5, truncate=False)

+--------------------+
|HV Ratio            |
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
+--------------------+
only showing top 5 rows



#### What day had the Peak High in Price?

In [ ]:
print(df.orderBy(df['High'].desc()).head(1)[0][0])

2015-01-13


#### What is the mean of the Close column?

In [ ]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [ ]:
from pyspark.sql.functions import max, min
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [ ]:
print(df.filter(df['Close'] < 60).count())

81


#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
print((df.filter(df['High']>80).count()/df.count()) * 100)

9.141494435612083


#### What is the Pearson correlation between High and Volume?

In [ ]:
from pyspark.sql.functions import corr
df.select(corr('High','Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [ ]:
from pyspark.sql.functions import year
yeardf = df.withColumn("Year", year(df['Date']))
max_df = yeardf.groupBy('Year').max()
max_df.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
from pyspark.sql.functions import month
monthdf = df.withColumn('Month',month('Date'))
monthavgs = monthdf.select(['Month','Close']).groupBy('Month').mean()
monthavgs.select('Month','avg(Close)').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great Job!